In [1]:
max_n_image = 192560
max_n_image_not_nan = 42560

In [2]:
from Model import SegNetModel, RetrainedClassificationModel
from HyperparameterOptimization import findOptimalBatchSize

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from math import log, ceil
import numpy as np
import keras

In [4]:
from keras.utils import plot_model
model = SegNetModel('noname', use_residual=True, use_argmax=True)
plot_model(model.model, to_file='model.png', show_shapes=True)

Done building encoder..
Done building decoder..


In [5]:
bs = 1 # findOptimalBatchSize(model, 2, save_excel=False)
model.train(bs, n_epoch=100000)
model.model.save('maxsized.hd5')
acc = model.eval(bs, remove_nan=False)
print(reduce_size, acc)

Epoch 1/100000


ResourceExhaustedError: OOM when allocating tensor with shape[2359296,4] and type int32 on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node max_unpooling2d_3/max_unpooling2d_3/transpose}} = Transpose[T=DT_INT32, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](max_unpooling2d_3/max_unpooling2d_3/Reshape_1/_821, max_unpooling2d_3/max_unpooling2d_3/transpose/sub_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss/mul/_887}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7869_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_u

In [ ]:
model = SegNetModel('noname')
bs = findOptimalBatchSize(model, 2, save_excel=False, reduced_size=1024)
acc_arr = []
hst_arr = []
for reduce_size in np.array(2) ** range(11, int(ceil(log(max_n_image_not_nan, 2))) + 1):
    reduce_size = min(reduce_size, max_n_image_not_nan)
    hst = model.train(bs, n_epoch=100000, reduced_size=reduce_size)
    hst_arr.append(hst)
    model.model.save(str(reduce_size) + '.hd5')
    acc = model.eval(bs, remove_nan=False)
    acc_arr.append(acc)
    print(reduce_size, acc)

In [ ]:
hst = model.train(bs, n_epoch=1000, remove_nan=False)
model.model.save('alldata.hd5')
acc.append(model.eval(bs, remove_nan=False))